In [1]:
%whos

Interactive namespace is empty.


In [2]:
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
from typing import Tuple
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# Please download this file from One Drive and test locally
# Show and explain all steps to pinpoint failure point
# Model must run or report exact failure point and specific plan to fix by 5pm
# write decoded samples to file, send to UMAP, upload to drive by 5pm, 9-30

In [ ]:
# Ideas:
# Run UMAP on working model, colors are by subtype
# must have labels

In [2]:
eight_by_five = pd.read_csv(
    '/Users/karlberb/work/sample_count/smpl_gen/architechture/X_intersect_5k_mad_feats_cohorts_n25.tsv',
    sep = '\t', index_col = 0)

In [3]:
ebf_y = pd.read_csv(
    '/Users/karlberb/work/sample_count/smpl_gen/architechture/y_intersect_5k_mad_feats_cohorts_n25.tsv',
    sep = '\t', index_col = 0)

In [4]:
ebf_y

,Labels
TCGA-OR-A5J1,ACC_2
TCGA-OR-A5J2,ACC_2
TCGA-OR-A5J3,ACC_3
TCGA-OR-A5J5,ACC_3
TCGA-OR-A5J6,ACC_1
...,...
TCGA-WC-AA9E,UVM_3
TCGA-YZ-A980,UVM_1
TCGA-YZ-A982,UVM_2
TCGA-YZ-A983,UVM_3


In [3]:
eight_by_five.shape

(8009, 5000)

In [4]:
type(eight_by_five)

pandas.core.frame.DataFrame

In [12]:
ebf_ary = np.array(eight_by_five)

In [13]:
ebf_ary.shape

(8009, 5000)

In [14]:
# Feed this to a model
inputs_ebf = keras.Input(shape=(5000))

In [15]:
inputs_ebf.shape

TensorShape([None, 5000])

In [16]:
type(inputs_ebf)

tensorflow.python.keras.engine.keras_tensor.KerasTensor

# fit() wants two data
# Build manually here
train_data  
val_data

In [11]:
%whos

Variable         Type          Data/Info
----------------------------------------
CSVLogger        type          <class 'tensorflow.python<...>ras.callbacks.CSVLogger'>
EarlyStopping    type          <class 'tensorflow.python<...>callbacks.EarlyStopping'>
MinMaxScaler     type          <class 'sklearn.preproces<...>sing._data.MinMaxScaler'>
Model            type          <class 'tensorflow.python<...>s.engine.training.Model'>
StandardScaler   type          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
Tuple            _TupleType    typing.Tuple
eight_by_five    DataFrame                   N:GEXP::COL<...>8009 rows x 5000 columns]
inspect          module        <module 'inspect' from '/<...>ib/python3.9/inspect.py'>
keras            module        <module 'tensorflow.keras<...>rflow/keras/__init__.py'>
layers           module        <module 'tensorflow.keras<...>eras/layers/__init__.py'>
normalize        function      <function normalize at 0x7fd460bbd8b0>
np               modu

### Normalization

In [5]:
# This is not a class so it is not inheriting anything
def normalize(data: pd.DataFrame,
              features: list, # In case labels are columns?
              feature_range: Tuple = None, # Please annotate
              scaler=None) -> (pd.DataFrame, any): # Please annotate
    
        # Describe logic here
        if feature_range is not None and scaler is None:
            scaler = MinMaxScaler(feature_range=feature_range)
            scaler.fit(data)
            
        elif feature_range is None and scaler is None:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler.fit(data)

        data = scaler.transform(data) # returns scaled data object
                                      # based on passed feature_range

        return pd.DataFrame(columns=features, data=data), scaler

In [6]:
type(normalize)

function

In [7]:
normalize.__doc__

In [8]:
import inspect

In [ ]:
# Is there fake data / test data to run this model on?

In [10]:
# Why is this function returning info about the real data?
inspect.getmembers(normalize)

[('__annotations__',
  {'data': pandas.core.frame.DataFrame,
   'features': list,
   'feature_range': typing.Tuple,
   'return': (pandas.core.frame.DataFrame, <function any(iterable, /)>)}),
 ('__call__',
  <method-wrapper '__call__' of function object at 0x7fd460bbd8b0>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object normalize at 0x7fd498518190, file "/var/folders/5v/88tvwqd94gs4dfr3h6rt1_w0qd13bg/T/ipykernel_96665/4120225083.py", line 1>),
 ('__defaults__', (None, None)),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x7fd460bbd8b0>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x7fd460bbd8b0>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x7fd460bbd8b0>),
 ('__get__', <method-wrapper '__get__' of function object at 0x7fd460bbd8b0>),
 ('__getattribute

In [ ]:
# End normalize inspections

### Split handler - remove for now, split manually above
### Do not need three spilts for UMAP

In [ ]:
def create_splits(input_data: pd.DataFrame,
                  without_val: bool = False) -> Tuple: # what is this syntax?
        """
        Creates train val test split of the data provided
        @param input_data: The input data which should be split
        @param without_val: If true only a test and a train set will be created
        @return: A tuple containing all the sets.
        """

        # No validation set will be created
        if without_val:
            return train_test_split(input_data, test_size=0.2, random_state=1, shuffle=True)

        # Create validation set
        X_dev, X_val = train_test_split(input_data, test_size=0.05, random_state=1, shuffle=True)
        X_train, X_test = train_test_split(X_dev, test_size=0.25, random_state=1, shuffle=True)
        return X_train, X_val, X_test

### Create custom sampling layer

In [20]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs): # Why no super?
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Create VAE 

In [19]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs): # What are the kwargs
        super(VAE, self).__init__(**kwargs) # Why super?
        self.encoder: Model = encoder
        self.decoder: Model = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property # property decorator, purpose?
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss_fn = keras.losses.MeanSquaredError()
            reconstruction_loss = reconstruction_loss_fn(data, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        return self.decoder(z)

### Adjust hyperparameters

In [18]:
latent_dim = 500 # Latent dims
# data_path = "" # Path to data

In [21]:
data_path = "/Users/karlberb/work/sample_count/smpl_gen/architechture/X_intersect_5k_mad_feats_cohorts_n25.tsv"

### Load data

In [22]:
data = pd.read_csv(data_path, sep='\t', index_col=0)

In [24]:
# Remove for now, manual split
train_data, val_data, test_data = SplitHandler.create_splits(input_data=data, without_val=False)

NameError: name 'SplitHandler' is not defined

In [ ]:
# Future work: put train test in function, SKIP UNTIL MODEL RUNS

# Do not run, model does not take 'test_data'
# GOAL IS TO MAKE A UMAP

data = pd.read_csv(data_path, sep='\t', index_col=0)
### truncate everything after the 4th decimal
data = np.trunc(1000 * data) / 1000 # Need a model that fits the data, skip

train_data, val_data, test_data = SplitHandler.create_splits(input_data=data, without_val=False)

train_data, scaler = normalize(train_data, features=train_data.columns)
val_data, _ = normalize(val_data, features=val_data.columns, scaler=scaler)
test_data, _ = normalize(data=test_data, features=test_data.columns, scaler=scaler)

### Build model

In [16]:
input_dimensions = train_data.shape[1]

encoder_inputs = keras.Input(shape=(input_dimensions,))
x = layers.Dense(units=input_dimensions / 2, activation="relu")(encoder_inputs)
x = layers.Dense(units=input_dimensions / 3, activation="relu")(x)
x = layers.Dense(units=input_dimensions / 4, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=input_dimensions / 4, activation="relu")(latent_inputs)
x = layers.Dense(units=input_dimensions / 3, activation="relu")(x)
x = layers.Dense(units=input_dimensions / 2, activation="relu")(x)

decoder_outputs = layers.Dense(units=input_dimensions, activation="relu")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

vae: VAE = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

# vae.summary()

callbacks = []

early_stop = EarlyStopping(monitor="reconstruction_loss",
                           mode="min", patience=5,
                           restore_best_weights=True)
callbacks.append(early_stop)

csv_logger = CSVLogger(os.path.join(base_path, 'training.log'),
                       separator='\t')
callbacks.append(csv_logger)

history = vae.fit(train_data,
                  callbacks=callbacks,
                  validation_data=(val_data, val_data),
                  epochs=500, batch_size=128)


NameError: name 'train_data' is not defined